# 文档向量化存储

之前的rag代码存在问题：每次提交问题时，都需要重新加载文档，然后向量化，这个流程是不太合理的。所以，我们需要把向量化之后的文档缓存下来，这样就不用每次都重新加载了。

首先沿用之前的加载分割文档的逻辑：

In [1]:
import shutup
shutup.please()
from dotenv import load_dotenv
load_dotenv()

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyMuPDFLoader

loader_about = PyMuPDFLoader('rag_example/about.pdf')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20
)
documents_about = loader_about.load_and_split(text_splitter)
documents_about

[Document(page_content='英雄联盟：双城之战》第二季简介\n《英雄联盟：双城之战》（Arcane）是拳头公司（Riot Games）与法国动画工作室Fortiche联合制作的\n原创动画剧集，基于英雄联盟的背景世界打造。第二季于2024年11月上线，共包含9集，继续讲述了皮\n尔特沃夫（Piltover）与祖安（Zaun）之间复杂的矛盾与冲突。\n剧情概述', metadata={'source': 'rag_example/about.pdf', 'file_path': 'rag_example/about.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.4', 'title': 'a', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/131.0.0.0 Safari/537.36', 'producer': 'Skia/PDF m131', 'creationDate': "D:20241129070925+00'00'", 'modDate': "D:20241129070925+00'00'", 'trapped': ''}),
 Document(page_content='剧情概述\n第二季聚焦蔚（Vi）与金克丝（Jinx）之间复杂的情感与对立关系，同时引入了更多英雄联盟宇宙的重\n要角色。剧中呈现了皮尔特沃夫和祖安两个城市在科技与魔法交织下的社会矛盾，以及在不断扩展的英\n雄联盟宇宙中关于权力与自由的讨论。\n角色与制作\n剧集主要角色包括：\n蔚（Vi）： 执着且勇敢的地下拳手。\n金克丝（Jinx）： 具有不稳定精神状态的疯狂发明家。', metadata={'source': 'rag_example/about.pdf', 'file_path': 'rag_example/about.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.4', 'title'

现在，我们需要把向量化文档存储在本地：

In [2]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS

embedding = OpenAIEmbeddings(model='text-embedding-ada-002')

documents_db = FAISS.from_documents(documents_about, embedding)
documents_db.save_local('vector_db')

存储在本地之后，就可以从存储中加载文件了：

In [3]:
documents_db = FAISS.load_local('./vector_db', embedding)
documents_db.similarity_search('剧集的主要角色有谁？', k=3)

[Document(page_content='剧情概述\n第二季聚焦蔚（Vi）与金克丝（Jinx）之间复杂的情感与对立关系，同时引入了更多英雄联盟宇宙的重\n要角色。剧中呈现了皮尔特沃夫和祖安两个城市在科技与魔法交织下的社会矛盾，以及在不断扩展的英\n雄联盟宇宙中关于权力与自由的讨论。\n角色与制作\n剧集主要角色包括：\n蔚（Vi）： 执着且勇敢的地下拳手。\n金克丝（Jinx）： 具有不稳定精神状态的疯狂发明家。', metadata={'source': 'rag_example/about.pdf', 'file_path': 'rag_example/about.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.4', 'title': 'a', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/131.0.0.0 Safari/537.36', 'producer': 'Skia/PDF m131', 'creationDate': "D:20241129070925+00'00'", 'modDate': "D:20241129070925+00'00'", 'trapped': ''}),
 Document(page_content='观众的高度评价。不过，大结局也引发了一些争议，部分观众对剧情发展表达了不同意见。', metadata={'source': 'rag_example/about.pdf', 'file_path': 'rag_example/about.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.4', 'title': 'a', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (

文档的增量更新：

In [4]:
len(documents_db.docstore._dict)

6

In [5]:
documents_db_more = FAISS.from_texts([
    '这是一段需要追加的新文本'
], embedding)
documents_db.merge_from(documents_db_more)
len(documents_db.docstore._dict)

7